In [1]:
#Import necessary libraries
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from tensorflow.keras import layers, models

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
#Load datasets
train_data = pd.read_csv('/kaggle/input/ecg-heartbeat-covid-19/heartbeat_125_lead_ii_train_validation_dataset.csv', header=None)
test_data = pd.read_csv('/kaggle/input/ecg-heartbeat-covid-19/heartbeat_125_lead_ii_test_dataset.csv', header=None)


In [4]:
X = train_data.iloc[:, :-1]
y = train_data.iloc[:, -1]
y = np.where(y == 4.0, 1, 0)

In [5]:
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
y_test = np.where(y_test==4, 1, 0)

In [6]:
covid_count = np.sum(y == 1)
non_covid_count = np.sum(y == 0)
# Use class weights to handle imbalance
class_weights = {0: len(y) / non_covid_count, 1: len(y) / covid_count}

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=4)

In [8]:
def resnet_block(inputs, filters, kernel_size=3, strides=1):
    x = layers.Conv1D(filters, kernel_size, strides=strides, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters, kernel_size, strides=1, padding="same")(x)
    x = layers.BatchNormalization()(x)

    shortcut = layers.Conv1D(filters, 1, strides=strides, padding="same")(inputs)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x

# Build ResNet model
input_dim = X_train.shape[1]
input_layer = layers.Input(shape=(input_dim, 1))
x = resnet_block(input_layer, 32)
x = resnet_block(x, 64)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

resnet_model = models.Model(input_layer, output_layer)


In [9]:
resnet_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[
    tf.keras.metrics.AUC(name="auc"),
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')    
])

In [10]:
# Ensure data is in NumPy array format before adding a new axis
X_train_resnet = np.array(X_train)[..., np.newaxis]
X_val_resnet = np.array(X_val)[..., np.newaxis]
X_test_resnet = np.array(X_test)[..., np.newaxis]


In [11]:
history = resnet_model.fit(X_train_resnet, y_train, epochs=10, batch_size=32, validation_data=(X_val_resnet, y_val), verbose=1)

Epoch 1/10


I0000 00:00:1740747376.592521     424 service.cc:145] XLA service 0x7c1df400a1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740747376.592579     424 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1740747376.592583     424 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


   52/39546 ━━━━━━━━━━━━━━━━━━━━ 1:59 3ms/step - accuracy: 0.7437 - auc: 0.5546 - loss: 0.5444 - precision: 0.2311 - recall: 0.1209 

I0000 00:00:1740747379.742922     424 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


39546/39546 ━━━━━━━━━━━━━━━━━━━━ 154s 4ms/step - accuracy: 0.8940 - auc: 0.9089 - loss: 0.2674 - precision: 0.8260 - recall: 0.5877 - val_accuracy: 0.9411 - val_auc: 0.9724 - val_loss: 0.1540 - val_precision: 0.9039 - val_recall: 0.7894
Epoch 2/10
39546/39546 ━━━━━━━━━━━━━━━━━━━━ 144s 4ms/step - accuracy: 0.9443 - auc: 0.9745 - loss: 0.1466 - precision: 0.9111 - recall: 0.7992 - val_accuracy: 0.9429 - val_auc: 0.9749 - val_loss: 0.1500 - val_precision: 0.9388 - val_recall: 0.7647
Epoch 3/10
39546/39546 ━━━━━━━━━━━━━━━━━━━━ 145s 4ms/step - accuracy: 0.9553 - auc: 0.9833 - loss: 0.1183 - precision: 0.9278 - recall: 0.8419 - val_accuracy: 0.9561 - val_auc: 0.9828 - val_loss: 0.1175 - val_precision: 0.9374 - val_recall: 0.8362
Epoch 4/10
39546/39546 ━━━━━━━━━━━━━━━━━━━━ 144s 4ms/step - accuracy: 0.9604 - auc: 0.9869 - loss: 0.1044 - precision: 0.9350 - recall: 0.8620 - val_accuracy: 0.9653 - val_auc: 0.9896 - val_loss: 0.0924 - val_precision: 0.9352 - val_recall: 0.8881
Epoch 5/10
39546/39

In [12]:
results = resnet_model.evaluate(X_test_resnet, y_test, verbose=0)
loss = results[0]
auc = results[1]
accuracy = results[2]
precision = results[3]
recall = results[4]
print(f"  - Loss: {loss:.4f}")
print(f"  - Precision: {precision:.4f}")
print(f"  - Recall: {recall:.4f}")
print(f"  - Accuracy: {accuracy:.4f}")
print(f"  - AUC: {auc:.4f}")

  - Loss: 0.0075
  - Precision: 0.9924
  - Recall: 0.9989
  - Accuracy: 0.9983
  - AUC: 0.9998


In [13]:
predictions = resnet_model.predict(X_test_resnet)

2806/2806 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [14]:
def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(history.history[metric_name],color='blue',label=metric_name)
    plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)

def plot_confusion_matrix(y_true, y_pred, title='', labels=[0,1]):
    cm = confusion_matrix(test_Y[1], np.round(type_pred), labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=[0, 1])
    disp.plot(values_format='d');

def plot_diff(y_true, y_pred, title = '' ):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.plot([-100, 100], [-100, 100])
    return plt


In [15]:
my_metrics = ["auc", "accuracy", 'precision', 'recall']